It might be useful to run 'pip3 install &lt;library&gt; --upgrade" to make sure you have latest versions of pandas, matplotlib, folium, etc.

In [26]:
# load libraries

import folium
from folium.plugins import MarkerCluster
import pandas as pd    # for working with "rows/columns" oriented data
import pymysql.cursors # 

# not using these right now, but I might...
import io  # useful routines for input/output

import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
         
import numpy as np



In [27]:
# i will email everyone this file. put it in same folder
# Connect to the database
db_params = open("mikepnyu_database_params.txt").read().split()

In [28]:
# Connect to the database

connection = pymysql.connect(host=db_params[0],
                             user=db_params[1],
                             password=db_params[2],
                             db=db_params[3],
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

In [29]:
# Noting that some are having problems with pd.read_sql, I'm stick with pymysql.
# But I move that quickly into a pandas datasframe.
# Try that with your more complex queries. Does that help?

sql = """
SELECT Libanius_Letters.Letter_ID, Libanius_Destinations.Destination_ID, Libanius_Destinations.Destination_Type, Libanius_Destinations.Latitude, 
Libanius_Destinations.Longitude
FROM Libanius_Letters, Libanius_Destinations
WHERE Libanius_Letters.Destination_ID = Libanius_Destinations.Destination_ID
ORDER BY Libanius_Letters.Letter_ID"""

# this is a pretty efficient code block for going from cursor to dataframe
with connection.cursor() as cursor:

    cursor.execute(sql)
    names = [ x[0] for x in cursor.description]
    result = cursor.fetchall()

destinations = pd.DataFrame(result, columns = names)

destinations['Total'] = destinations.groupby(['Destination_ID'])['Destination_ID'].transform('count')

# df # uncommanet at start of line if you want to see contents

In [30]:
import folium

libanius_letters = folium.Map(location=[35, 22], zoom_start=4,tiles='http://{s}.tile.openstreetmap.se/hydda/base/{z}/{x}/{y}.png',
                   attr='Tiles courtesy of <a href="http://openstreetmap.se/" target="_blank">OpenStreetMap Sweden</a> &mdash; Map data &copy; <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a>')

galatia_geo = '.\Maps\map_Galatia.geojson'
egypt_geo = '.\Maps\map_Egypt2.geojson'
cappadocia_geo = '.\Maps\map_Cappadocia.geojson'
bithynia_geo = '.\Maps\map_Bithynia3.geojson'
armenia_geo = '.\Maps\map_Armenia.geojson'
cilicia_geo = '.\Maps\map_Cilicia.geojson'
euphratensis_geo = '.\Maps\map_Euphratensis.geojson'
palestine_geo = '.\Maps\map_Palestine.geojson'
phoenicia_geo = '.\Maps\map_Phoenicia2.geojson'
arabia_geo = '.\Maps\map_Arabia.geojson'

libanius_letters.choropleth(geo_path=galatia_geo, fill_color="#FF0000", fill_opacity=0.8)
libanius_letters.choropleth(geo_path=egypt_geo, fill_color="#CC00FF", fill_opacity=0.8)
libanius_letters.choropleth(geo_path=cappadocia_geo, fill_color="#CC00FF", fill_opacity=0.8)
libanius_letters.choropleth(geo_path=bithynia_geo, fill_color="#CC00FF", fill_opacity=0.8)
libanius_letters.choropleth(geo_path=armenia_geo, fill_color="#0000FF", fill_opacity=0.8)
libanius_letters.choropleth(geo_path=cilicia_geo, fill_color="#CC00FF", fill_opacity=0.8)
libanius_letters.choropleth(geo_path=euphratensis_geo, fill_color="#FF0000", fill_opacity=0.8)
libanius_letters.choropleth(geo_path=palestine_geo, fill_color="#CC00FF", fill_opacity=0.8)
libanius_letters.choropleth(geo_path=phoenicia_geo, fill_color="#FF0000", fill_opacity=0.8)
libanius_letters.choropleth(geo_path=arabia_geo, fill_color="#CC00FF", fill_opacity=0.8)

for index, row in destinations.iterrows():
    if row["Destination_Type"] == "City":
        if row["Total"] == 1:
            folium.Marker([row["Latitude"], row["Longitude"]], popup=row["Destination_ID"]+ ", 1 Letter : F" + str(row["Letter_ID"]), 
                      icon = folium.Icon(icon = 'pushpin', color ='blue')).add_to(libanius_letters)
        elif row["Total"] >= 2 and row["Total"] < 5:
            folium.Marker([row["Latitude"], row["Longitude"]], popup=row["Destination_ID"]+ ", " + str(row["Total"]) + " Letters", 
                      icon = folium.Icon(icon = 'pushpin', color ='purple')).add_to(libanius_letters)
        else:
            folium.Marker([row["Latitude"], row["Longitude"]], popup=row["Destination_ID"]+ ", " + str(row["Total"]) + " Letters", 
                      icon = folium.Icon(icon = 'pushpin', color ='red')).add_to(libanius_letters)
    if row["Destination_Type"] == "Region":        
        if row["Total"] > 0 and row["Total"] < 2:
            folium.Marker([row["Latitude"], row["Longitude"]], popup=row["Destination_ID"]+ ", 1 Letter : F" + str(row["Letter_ID"]), 
                      icon = folium.Icon(icon = 'question-sign', color ='blue')).add_to(libanius_letters)
        elif row["Total"] >= 2 and row["Total"] < 5:
            folium.Marker([row["Latitude"], row["Longitude"]], popup=row["Destination_ID"]+ ", " + str(row["Total"]) + " Letters", 
                      icon = folium.Icon(icon = 'question-sign', color ='purple')).add_to(libanius_letters)
        else:
            folium.Marker([row["Latitude"], row["Longitude"]], popup=row["Destination_ID"]+ ", " + str(row["Total"]) + " Letters", 
                      icon = folium.Icon(icon = 'question-sign', color ='red')).add_to(libanius_letters)
            
folium.Marker([36.20138, 36.161749], popup="Antioch, Home of Libanius and Origin of the Letters",icon = folium.Icon(icon = 'fullscreen', color ='green')).add_to(libanius_letters)

libanius_letters

In [31]:
import folium
import pandas as pd

sql2 = """
SELECT Libanius_Destinations.Destination_ID
FROM Libanius_Letters, Libanius_Destinations
WHERE (Libanius_Letters.Destination_ID = Libanius_Destinations.Destination_ID)
    AND (Libanius_Destinations.Destination_Type = "Region")
ORDER BY Libanius_Destinations.Destination_ID"""

# this is a pretty efficient code block for going from cursor to dataframe
with connection.cursor() as cursor:

    cursor.execute(sql2)
    names = [ x[0] for x in cursor.description]
    result = cursor.fetchall()

destinations_choropleth = pd.DataFrame(result, columns = names)

destinations_choropleth['Total'] = destinations_choropleth.groupby(['Destination_ID'])['Destination_ID'].transform('count')

destinations_choropleth.to_csv('Choropleth\destinations_choropleth.csv')

provinces_geo = 'Choropleth\map_provinces.geojson'
provinces_letters = 'Choropleth\destinations_choropleth.csv'

provinces_data = pd.read_csv(provinces_letters)

libanius_letters3 = folium.Map(location=[35, 22], zoom_start=4,tiles='http://{s}.tile.openstreetmap.se/hydda/base/{z}/{x}/{y}.png',
                   attr='Tiles courtesy of <a href="http://openstreetmap.se/" target="_blank">OpenStreetMap Sweden</a> &mdash; Map data &copy; <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a>')

libanius_letters3.choropleth(geo_path=provinces_geo,
                             data_out='data1.json',
                             data=provinces_data,
                             columns=['Destination_ID', 'Total'],
                             key_on='feature.properties.name',
                             threshold_scale=[0, 2, 4, 6, 8, 16],
                             fill_color='YlOrRd', fill_opacity=0.7, line_opacity=0.5,
                             legend_name="Number of Letters per Province")

for index, row in destinations.iterrows():
    if row["Destination_Type"] == "City":
        if row["Total"] == 1:
            folium.Marker([row["Latitude"], row["Longitude"]], popup=row["Destination_ID"]+ ", 1 Letter : F" + str(row["Letter_ID"]), 
                      icon = folium.Icon(icon = 'pushpin', color ='blue')).add_to(libanius_letters3)
        elif row["Total"] >= 2 and row["Total"] < 5:
            folium.Marker([row["Latitude"], row["Longitude"]], popup=row["Destination_ID"]+ ", " + str(row["Total"]) + " Letters", 
                      icon = folium.Icon(icon = 'pushpin', color ='purple')).add_to(libanius_letters3)
        else:
            folium.Marker([row["Latitude"], row["Longitude"]], popup=row["Destination_ID"]+ ", " + str(row["Total"]) + " Letters", 
                      icon = folium.Icon(icon = 'pushpin', color ='red')).add_to(libanius_letters3)
    if row["Destination_Type"] == "Region":        
        if row["Total"] > 0 and row["Total"] < 2:
            folium.Marker([row["Latitude"], row["Longitude"]], popup=row["Destination_ID"]+ ", 1 Letter : F" + str(row["Letter_ID"]), 
                      icon = folium.Icon(icon = 'question-sign', color ='blue')).add_to(libanius_letters3)
        elif row["Total"] >= 2 and row["Total"] < 5:
            folium.Marker([row["Latitude"], row["Longitude"]], popup=row["Destination_ID"]+ ", " + str(row["Total"]) + " Letters", 
                      icon = folium.Icon(icon = 'question-sign', color ='purple')).add_to(libanius_letters3)
        else:
            folium.Marker([row["Latitude"], row["Longitude"]], popup=row["Destination_ID"]+ ", " + str(row["Total"]) + " Letters", 
                      icon = folium.Icon(icon = 'question-sign', color ='red')).add_to(libanius_letters3)

folium.Marker([36.20138, 36.161749], popup="Antioch, Home of Libanius and Origin of the Letters",icon = folium.Icon(icon = 'fullscreen', color ='green')).add_to(libanius_letters3)


libanius_letters3

In [32]:
# connection.commit()
# connection.close()